In [ ]:
import tensorflow as tf
import numpy as np

# Data Preprocessing 

In [ ]:
import os.path as osp
import sys
sys.path.append("/home/service/")
from ThreeDLAPGAN.external.LGan.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from ThreeDLAPGAN.external.LGan.src.autoencoder import Configuration as Conf
from ThreeDLAPGAN.external.LGan.src.point_net_ae import PointNetAutoEncoder

from  ThreeDLAPGAN.external.LGan.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder
from  ThreeDLAPGAN.external.LGan.src.in_out import  PointCloudDataSet


from ThreeDLAPGAN.external.LGan.src.tf_utils import reset_tf_graph
import tqdm
import plotly

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.

experiment_name = 'single_class_ae'
n_pc_points = 2048                              # Number of points per model.
bneck_size = 128                                # Bottleneck-AE size
ae_loss = 'emd'                             # Loss to optimize: 'emd' or 'chamfer'
class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()

In [ ]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
print(class_dir)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

In [ ]:
# all_pc_data.point_clouds[0].shape

## Make downsampling 

### Fool Downsampling

In [ ]:
pc_count = int(raw_input('Give me the count of points you want to be in point_cloud e.g. 1024: ').lower())
assert pc_count <= all_pc_data.n_points

In [ ]:
pclouds = np.empty([all_pc_data.point_clouds.shape[0],pc_count, all_pc_data.point_clouds.shape[2]], dtype=np.float32)
for i,pc in enumerate(all_pc_data.point_clouds):
    mask = np.zeros(p
                    c.shape[0])
    a = np.random.choice(pc.shape[0],size = pc_count,replace=False)
    mask[a] = 1
    pclouds[i] = pc[mask.astype(bool)]

### Smart downsampling 

In [ ]:
print(all_pc_data.num_examples)

In [ ]:
# pclouds_smart = np.empty([all_pc_data.point_clouds.shape[0],pc_count, all_pc_data.point_clouds.shape[2]], dtype=np.float32)
from sklearn.cluster import KMeans
from tqdm import tqdm
def get_smart_downsample(data,pc_count):
    buf_size=1
    fout = open('log.txt', 'wb', buf_size)
    pclouds_smart = np.empty([data.shape[0],pc_count, data.shape[2]], dtype=np.float32)
    for i,pc in tqdm(enumerate(data)):
        while True:
            try:
                fout.write(str(i)+'\n')
                kmeans  = KMeans(n_clusters = pc_count,n_jobs=-1)
                kmeans.fit(pc)
                pclouds_smart[i] = kmeans.cluster_centers_ 
                break
            except:
                continue
    fout.close()
    return pclouds_smart

In [ ]:
# pclouds_smart512 = get_smart_downsample(pclouds_smart1024,512)

## Save and load pickled Data

In [ ]:
import pprint, pickle
def read_pickle_data(input='data.pkl'):
    pkl_file = open(input, 'rb')

    pclouds = pickle.load(pkl_file)

    pkl_file.close()
    return pclouds

def pickle_data(pcloud_smart=None,output='data.pkl'):
    output = open(output, 'wb')
    pickle.dump(pcloud_smart, output)
    output.close()

In [ ]:
# pickle_data(all_pc_data.point_clouds,'pclouds_smart2048.pkl')

In [ ]:
# pclouds_smart.shape

In [ ]:
pclouds_smart2048 = read_pickle_data(input='pclouds_smart2048.pkl')
pclouds_smart1024 = read_pickle_data(input='pclouds_smart1024.pkl')
pclouds_smart512 = read_pickle_data(input='pclouds_smart512.pkl')

In [ ]:
# pclouds_smart2048 = all_pc_data.point_clouds

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
import plotly.offline as offline

x, y, z = pclouds_smart2048[0,:,0],pclouds_smart2048[0,:,1],pclouds_smart2048[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='smart_downsamplingv_2048')

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
import plotly.offline as offline

x, y, z = pclouds_smart[0,:,0],pclouds_smart[0,:,1],pclouds_smart[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='smart_downsamplingv_512')

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

plotly.tools.set_credentials_file(username='Vahe1994', api_key='MTEFahTX43GTctGnrfox')

x, y, z = pclouds[0,:,0],pclouds[0,:,1],pclouds[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

## Checking Resul from downsampling

In [ ]:
x, y, z = pclouds[0,:,0],pclouds[0,:,1],pclouds[0,:,2]

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

plotly.tools.set_credentials_file(username='Vahe1994', api_key='MTEFahTX43GTctGnrfox')

x, y, z = pclouds[0,:,0],pclouds[0,:,1],pclouds[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

In [ ]:
minim_pc_data = PointCloudDataSet(pclouds)

In [ ]:
n_pc_points = 1024
train_dir = create_dir(osp.join(top_out_dir, experiment_name))
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)

In [ ]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).
conf.save(osp.join(train_dir, 'configuration'))

In [ ]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

In [ ]:
buf_size = 1 # flush each line
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(minim_pc_data, conf, log_file=fout)
fout.close()

In [ ]:
minim_pc_data.point_clouds.shape

In [ ]:
feed_pc, feed_model_names, _ = minim_pc_data.next_batch(10)
reconstructions = ae.reconstruct(feed_pc)
latent_codes = ae.transform(feed_pc)

In [ ]:
latent_codes.shape

In [ ]:
points = ae.decode(latent_codes)

In [ ]:

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

In [ ]:

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
plotly.tools.set_credentials_file(username='Vahe1994', api_key='MTEFahTX43GTctGnrfox')

x, y, z = points[0,:,0],points[0,:,1],points[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

In [ ]:
points.shape

### Let's look how work decoder on when input is random distrbution 

In [ ]:
points.shape

In [ ]:
points = ae.decode(np.abs(np.random.normal(0, 1, (10, 128))))

In [ ]:
x, y, z = points[0,:,0],points[0,:,1],points[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

As we can see nothing meaningfull 

### Restore AE

In [ ]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)
ae.restore_model(model_path='/home/service/ThreeDLAPGAN/data/single_class_ae/',epoch=500)

In [ ]:

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
plotly.tools.set_credentials_file(username='Vahe1994', api_key='MTEFahTX43GTctGnrfox')

x, y, z = points[0,:,0],points[0,:,1],points[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='drom restored-3d-scatter')

### Experiment with latent pace,that show smoothness of space

We took two random smaples from training sets get their vector's in latent space. After that we  construct mean of this two vectors and decode it. As you can see  it's very much alike of chair.

In [ ]:
points= []

latent_codes[2] = (latent_codes[0]+latent_codes[1])/2
points = ae.decode(latent_codes)

In [ ]:
x, y, z = points[0,:,0],points[0,:,1],points[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='first')

In [ ]:
x, y, z = points[1,:,0],points[1,:,1],points[1,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='second')

In [ ]:
x, y, z = points[2,:,0],points[2,:,1],points[2,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='merged')

## GAN training

In [ ]:
from ThreeDLAPGAN.src.generator_discriminator import discriminator_smpl,generator_smpl
from ThreeDLAPGAN.external.LGan.src.latent_gan import LatentGAN

In [ ]:
feed_pc, feed_model_names, _ = minim_pc_data.next_batch(1200)
reconstructions = ae.reconstruct(feed_pc)
latent_codes = ae.transform(feed_pc)

In [ ]:
experiment_name = 'GAN' 

In [ ]:
train_dir = create_dir(osp.join(top_out_dir, experiment_name))
train_params = default_train_params()
# generator, discriminator, generator_args, discriminator_args = Simple_Gan(128)
generator = generator_smpl
discriminator = discriminator_smpl

In [ ]:
reset_tf_graph()
gan = LatentGAN(name='latent_gan', learning_rate=0.0001, n_output=[128], noise_dim=128, discriminator=discriminator, generator=generator)

In [ ]:
lc = PointCloudDataSet(latent_codes)

In [ ]:
noise_params = {  'mu':0, 'sigma':1}

In [ ]:
buf_size = 1 # flush each line
train_stats=[]
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
for i in range(1000):
    feed_pc, feed_model_names, _ = minim_pc_data.next_batch(1000)
    reconstructions = ae.reconstruct(feed_pc)
    latent_codes = ae.transform(feed_pc)
    lc = PointCloudDataSet(latent_codes)
    train_stats=gan._single_epoch_train(lc,  batch_size=500, noise_params=noise_params)
    if(i%5==0):
        print(train_stats)
        train_stats=[]
fout.close()

In [ ]:
noise_params = {  'mu':0, 'sigma':1}
p = gan.generate(1,noise_params=noise_params)

In [ ]:
points =ae.decode(p)

In [ ]:
x, y, z = points[0,:,0],points[0,:,1],points[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='generated_v2')

In [ ]:
x, y, z = points[0,:,0],points[0,:,1],points[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='generated_v3')

In [ ]:
x, y, z = points[0,:,0],points[0,:,1],points[0,:,2]
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='generated')

In [ ]:
tflearn.fully_connected(layer, layer_sizes[n_layers - 1][0], activation='relu', weights_init='xavier', name=name, weight_decay=weight_decay, reuse=reuse, scope=scope_i)

In [ ]:
gan.save_model(3000)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from  plotly.offline import iplot,init_notebook_mode
import numpy as np
from sklearn.neighbors import NearestNeighbors

# plotly.tools.set_credentials_file(username='Vahe1994', api_key='MTEFahTX43GTctGnrfox')
def get_plot(pclouds=None,id_of_obj = 0,name = 'chair',mode=False):
    init_notebook_mode(mode)
    x, y, z = pclouds[id_of_obj,:,0],pclouds[id_of_obj,:,1],pclouds[id_of_obj,:,2]
    trace1 = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=4,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5
            ),
            opacity=0.8
        )
    )


    data = [trace1]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )
    fig = go.Figure(data=data, layout=layout)
    if mode:
        return py.iplot(fig, filename= name)
    else:
        return iplot(fig, filename= name)
# def add_points(pclouds,count):
#     for pc in pclouds:
        
def get_conf(experiment_name = 'AE2048',n_pc_points = 2048,bneck_size = 128,epoch=500 ):
    top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
    top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.
    ae_loss = 'emd'                             # Loss to optimize: 'emd' or 'chamfer'
    train_dir = create_dir(osp.join(top_out_dir, experiment_name))
    train_params = default_train_params()
    encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)
    conf = Conf(n_input = [n_pc_points, 3],
                loss = ae_loss,
                training_epochs = epoch,
                batch_size = train_params['batch_size'],
                denoising = train_params['denoising'],
                learning_rate = train_params['learning_rate'],
                train_dir = train_dir,
                loss_display_step = train_params['loss_display_step'],
                saver_step = train_params['saver_step'],
                z_rotate = train_params['z_rotate'],
                encoder = encoder,
                decoder = decoder,
                encoder_args = enc_args,
                decoder_args = dec_args
               )
    conf.experiment_name = experiment_name
    conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).
    conf.save(osp.join(train_dir, 'configuration'))
    return conf


In [ ]:
def Train_AE(train_ae = False,data = None,conf=None,URL = '/home/service/ThreeDLAPGAN/data/single_class_ae/'):
    if train_ae:
        reset_tf_graph()
        ae = PointNetAutoEncoder(conf.experiment_name, conf)
        buf_size = 1 # flush each line
        all_pc_data  = PointCloudDataSet(data)
        fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
        train_stats = ae.train(all_pc_data, conf, log_file=fout)
        fout.close()
    else:
        reset_tf_graph()
        ae = PointNetAutoEncoder(conf.experiment_name, conf)
        ae.restore_model(model_path=URL,epoch=epoch)
    return ae
def get_latent(ae,feed_pc):
    reconstructions = ae.reconstruct(feed_pc)
    latent_codes = ae.transform(feed_pc)
    return latent_codes
def Train_GAN(experiment_name = 'GAN1',gan=None,generator=None,discriminator=None,data=None):
    buf_size = 1 # flush each line
    train_stats=[]
    top_out_dir = '../data/'
    train_dir = create_dir(osp.join(top_out_dir, experiment_name))
    fout = open(osp.join(train_dir, 'train_stats.txt'), 'wb', buf_size)
    minim_pc_data = PointCloudDataSet(data)
    for i in range(1000):
        feed_pc, feed_model_names, _ = minim_pc_data.next_batch(1000)
        reconstructions = ae.reconstruct(feed_pc)
        latent_codes = ae.transform(feed_pc)
        lc = PointCloudDataSet(latent_codes)
        train_stats=gan._single_epoch_train(lc,  batch_size=500, noise_params=noise_params)
        if(i%20==0):
            clear_output()
            fout.write(train_stats)
            print(train_stats)
    fout.close()
    return gan
def Restore_AE(name ='AE512',epoch = 500,conf=None):
    top_dir ='../data/'
    direct =top_dir+name +'/'
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)
    ae.restore_model(model_path=direct,epoch=500)
    return ae

def get_smart_upsampling(data,pc_count,num_neighbors=5):
    buf_size=1
    fout = open('log.txt', 'wb', buf_size)
    pclouds_smart = np.empty([data.shape[0],data.shape[1]+pc_count, data.shape[2]], dtype=np.float32)
    for i,pc in tqdm(enumerate(data)):
        while True:
            try:
                fout.write(str(i)+'\n')
                knn  = NearestNeighbors(n_neighbors=num_neighbors)

                knn.fit(pc)

                neighbors = knn.kneighbors(pc, return_distance=False)
                pclouds_smart[i,:pc_count] = pc
                pclouds_smart[i,pc_count:] = pc[knn.kneighbors(pc, return_distance=False)].mean(axis=1)
                break
            except:
                continue
    fout.close()
    return pclouds_smart

In [ ]:
pclouds_smart2048[:int(2.0/3.0*pclouds_smart2048.shape[0])].shape

In [ ]:
ae2048 = Train_AE(train_ae = True,data = pclouds_smart2048[:int(2.0/3.0*pclouds_smart2048.shape[0])],conf=get_conf(experiment_name = 'AE2048_v5_0.6',n_pc_points = 2048,bneck_size = 128,epoch=1000),URL = None)

In [ ]:
ae1024 = Train_AE(train_ae = True,data = pclouds_smart1024[:int(2.0/3.0*pclouds_smart1024.shape[0])],conf=get_conf(experiment_name = 'AE1024_v5_0.6',n_pc_points = 1024,bneck_size = 128,epoch=1000),URL = None)

In [ ]:
# ae1024 = Train_AE(train_ae = True,data = pclouds_smart1024[:pclouds_smart1024.shape[0]/2],conf=get_conf(experiment_name = 'AE1024',n_pc_points = 1024,bneck_size = 128,epoch=400),URL = None)

In [ ]:
train_params = default_train_params()
# ae512 = Restore_AE(name ='AE512',epoch = 700,conf = get_conf(experiment_name='AE512',n_pc_points=512,epoch=700))
# ae1024 = Restore_AE(name ='AE1024',epoch = 700,conf = get_conf(experiment_name='AE1024',n_pc_points=1024,epoch=700))
ae2048 = Restore_AE(name ='AE2048',epoch = 500,conf = get_conf(experiment_name='AE2048',n_pc_points=2048,epoch=700))

In [ ]:
# pclouds = ae2048.decode(lt)
a= pclouds_smart2048[:2]

In [ ]:
a

In [ ]:
# lt = get_latent(ae2048,a)
lt= get_latent(ae2048,pclouds_smart2048[:4])

pclouds = ae2048.decode(lt)


In [ ]:
ae1024.decode(lt)

In [ ]:
pclouds[139]=ae512.decode(3.0/4*lt[600]+lt[78]*1.0/4)

In [ ]:
get_plot(pclouds=pclouds,id_of_obj=0)

In [ ]:
get_plot(pclouds=pclouds,id_of_obj=0)

In [ ]:
get_plot(pclouds=pclouds,id_of_obj=139)

In [ ]:
knn = NearestNeighbors(n_neighbors=4)
knn.fit(pclouds_smart512[1])

In [ ]:
pclouds_smart512[0].shape

In [ ]:
knn.kneighbors(pclouds_smart512[0], return_distance=False)[:,1:]

In [ ]:

a = pclouds_smart512[0][knn.kneighbors(pclouds_smart512[0], return_distance=False)].mean(axis=1)

In [ ]:
pclouds_smart512[0][knn.kneighbors(pclouds_smart512[0], return_distance=False)[1]].mean(axis=0)

In [ ]:
ls = get_smart_upsampling(pclouds,1024,neighbors=3)

In [ ]:
ls[0][:,0].sort()

In [ ]:
np.sort(ls[0],axis=0)

In [ ]:
get_plot(pclouds=ls,id_of_obj=0)

In [ ]:
get_plot(pclouds=ls,id_of_obj=0)

In [ ]:
pclouds_smart512.shape

In [ ]:
lt = get_latent(ae=ae512,feed_pc=pclouds_smart512[:5000])

k = get_latent(ae=ae512,feed_pc=pclouds_smart512[5000:])

lt512 = (np.concatenate((lt,k),axis=0))

In [ ]:
import numpy as np
import time
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
%matplotlib inline
TASK=2
def sample_noise(N,NOISE_DIM = 128):
    return np.random.normal(size=(N,NOISE_DIM)).astype(np.float32)
class Generator(nn.Module):
    def __init__(self, noise_dim, out_dim, hidden_dim=128):
        super(Generator, self).__init__()
        
        self.fc1 = nn.Linear(noise_dim, hidden_dim)
        nn.init.xavier_normal(self.fc1.weight)
        nn.init.constant(self.fc1.bias, 0.0)
        
        self.fc2 = nn.Linear(hidden_dim, out_dim)
        nn.init.xavier_normal(self.fc2.weight)
        nn.init.constant(self.fc2.bias, 0.0)
        

    def forward(self, z):
        """
            Generator takes a vector of noise and produces sample
        """
        h1 = F.relu(self.fc1(z))
        y_gen = self.fc2(h1)
        return y_gen
class Discriminator(nn.Module):
    def __init__(self, in_dim, hidden_dim=100):
        super(Discriminator, self).__init__()
        
        self.fc1 = nn.Linear(in_dim, 256)
        nn.init.xavier_normal(self.fc1.weight)
        nn.init.constant(self.fc1.bias, 0.0)
        
        self.fc2 = nn.Linear(256, 512)
        nn.init.xavier_normal(self.fc2.weight)
        nn.init.constant(self.fc2.bias, 0.0)
        
        self.fc3 = nn.Linear(512, 1)
        nn.init.xavier_normal(self.fc3.weight)
        nn.init.constant(self.fc3.bias, 0.0)
        

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        if TASK == 1 or TASK ==2:
            score = F.sigmoid(self.fc3(h2))
        else:
            score = self.fc3(h2)
        return score
class UnitNormClipper(object):

    def __init__(self, frequency=5):
        self.frequency = frequency

    def __call__(self, module):
        # filter the variables to get the ones you want
        if hasattr(module, 'weight'):
            w = module.weight.data

            w.clamp_(-0.001,0.001)
def g_loss(input_1):
    # if TASK == 1: 
    #     do something
    if TASK ==1 :
        return torch.mean(torch.log(1-input_1)).cuda()
    if TASK == 2:
        return -torch.mean(torch.log(input_1)).cuda()
    if TASK == 3 or TASK == 4:
        return -torch.mean(input_1)
    return # TODO
def d_loss(input_1,input_2,penalty =None) :
    if TASK == 1 or TASK == 2: 
        return -torch.mean(torch.log(input_2)).cuda() - torch.mean(torch.log(1-input_1)).cuda()

    if TASK==3 or TASK == 4:
        if(penalty is None):
            return -torch.mean(input_2).cuda() + torch.mean(input_1).cuda()
        else:
            return -torch.mean(input_2).cuda() + torch.mean(input_1).cuda() + penalty.cuda()
################################
def iterate_minibatches(X, batchsize, y=None):
    perm = np.random.permutation(X.shape[0])
    
    for start in range(0, X.shape[0], batchsize):
        end = min(start + batchsize, X.shape[0])
        if y is None:
            yield X[perm[start:end]]
        else:
            yield X[perm[start:end]], y[perm[start:end]]
def iterate_minibatches_with_inf(X,Inf, batchsize, y=None):
    perm = np.random.permutation(X.shape[0])
    
    for start in range(0, X.shape[0], batchsize):
        end = min(start + batchsize, X.shape[0])
        if y is None:
            yield X[perm[start:end]],Inf[perm[start:end]]
        else:
            yield X[perm[start:end]], y[perm[start:end]]

In [ ]:
generator_512 = Generator(128, out_dim = 128).cuda()
discriminator = Discriminator(in_dim = 128).cuda()

In [ ]:
from IPython import display


num_epochs = 200
batch_size = 50
learning_rate=0.0001

# ===========================
# IMPORTANT PARAMETER:
# Number of D updates per G update
# ===========================
k_d, k_g = 1, 1
if TASK ==3 or TASK==4 or TASK==2 or Task==1:
#     g_optimizer = optim.RMSprop(generator.parameters(), lr=1e-4)
#     d_optimizer = optim.RMSprop(discriminator.parameters(), lr=1e-4)
    lr =1e-4
    g_optimizer = optim.Adam(generator_512.parameters(),lr=lr)
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
    k_d = 1
    k_g = 1
accs = []
clipper = UnitNormClipper()
use_cuda=True
data = lt512
try:
    for epoch in range(num_epochs):
        ls_g=[]
        ls_d=[]
        for input_data in iterate_minibatches(data, batch_size):
            
            # Optimize D
            
            for _ in range(k_d):
                # Sample noise
                noise = Variable(torch.Tensor(sample_noise(len(input_data))).cuda())
                
                # Do an update
             
                inp_data = Variable(torch.Tensor(input_data).cuda())
                data_gen = generator_512(noise)
                if(TASK==4):
                    #COde from here https://github.com/EmilienDupont/wgan-gp
                    alpha = torch.rand(batch_size, 1,)
                    alpha = alpha.expand_as(inp_data)
                    if use_cuda:
                        alpha = alpha.cuda()
                    interpolated = alpha * inp_data.data + (1 - alpha) * data_gen.data
                    interpolated = Variable(interpolated, requires_grad=True)
                    if use_cuda:
                        interpolated = interpolated.cuda()

                    # Calculate probability of interpolated examples
                    prob_interpolated = discriminator(interpolated)

                    # Calculate gradients of probabilities with respect to examples
                    gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                                           grad_outputs=torch.ones(prob_interpolated.size()).cuda() if use_cuda else torch.ones(
                                           prob_interpolated.size()),
                                           create_graph=True, retain_graph=True)[0]

                    # Gradients have shape (batch_size, num_channels, img_width, img_height),
                    # so flatten to easily take norm per example in batch
                    gradients = gradients.view(batch_size, -1)
                   

                    # Derivatives of the gradient close to 0 can cause problems because of
                    # the square root, so manually calculate norm and add epsilon
                    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

                    # Return gradient penalty
                    penalty = 10 * ((gradients_norm - 1) ** 2).mean()
                    loss = d_loss(discriminator(data_gen), discriminator(inp_data),penalty)
                else:
                    loss = d_loss(discriminator(data_gen), discriminator(inp_data))
                ls_d.append(loss.data.cpu().numpy()[0])
                d_optimizer.zero_grad()
                loss.backward()
                d_optimizer.step()
                if TASK == 3:
                    discriminator.apply(clipper)

    
            # Optimize G
            for _ in range(k_g):
                # Sample noise
                noise = Variable(torch.Tensor(sample_noise(len(input_data))).cuda())
                
                # Do an update
                data_gen = generator_512 (noise)
                loss = g_loss(discriminator(data_gen))
                ls_g.append(loss.data.cpu().numpy()[0])
                g_optimizer.zero_grad()
                loss.backward()
                g_optimizer.step()
        if(epoch%10==0):
            print('generator_loss:',np.mean(ls_g),'discriminator_loss',np.mean(ls_d))
        
except KeyboardInterrupt:
    pass

In [ ]:
get_plot(ae2048.decode(get_latent(ae2048,pclouds_smart2048[:140])),0)

In [ ]:
get_plot(pclouds_smart2048,0)

In [ ]:
data_gen = generator(noise)

In [ ]:
generator

In [ ]:
data_gen.data.cpu().numpy().shape

In [ ]:
pclouds = ae512.decode(data_gen.data.cpu().numpy()) 

# Second level

In [ ]:
generator = Generator(256, out_dim = 128).cuda()
discriminator = Discriminator(in_dim = 128).cuda()

In [ ]:
pc_ups = get_smart_upsampling(pclouds_smart512[pclouds_smart512.shape[0]/2:],512,num_neighbors=4)

In [ ]:
lt = get_latent(ae=ae1024,feed_pc=pclouds_smart1024[:1000])
for i in np.arange(1000,pclouds_smart1024.shape[0],1000):
    lt =(np.concatenate((lt,get_latent(ae=ae1024,feed_pc=pclouds_smart1024[i:i+1000])),axis=0)) 
lt1024 = lt[pclouds_smart1024.shape[0]/2:]

In [ ]:
pc_ups[:1000].shape

In [ ]:
lt = get_latent(ae=ae1024,feed_pc=pc_ups[:1000])
for i in np.arange(1000,pc_ups.shape[0],1000):
    lt =(np.concatenate((lt,get_latent(ae=ae1024,feed_pc=pc_ups[i:i+1000])),axis=0)) 
lt_ups1024 = lt

In [ ]:
print(lt1024.shape)
print(inform.shape)

In [ ]:
from IPython import display


num_epochs = 100
batch_size = 50
learning_rate=0.0001

# ===========================
# IMPORTANT PARAMETER:
# Number of D updates per G update
# ===========================
k_d, k_g = 1, 1
if TASK ==3 or TASK==4 or TASK==2 or Task==1:
#     g_optimizer = optim.RMSprop(generator.parameters(), lr=1e-4)
#     d_optimizer = optim.RMSprop(discriminator.parameters(), lr=1e-4)
    lr =1e-4
    g_optimizer = optim.Adam(generator.parameters(),lr=lr)
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
    k_d = 1
    k_g = 1
accs = []
clipper = UnitNormClipper()
use_cuda=True
data = lt1024
inform = lt_ups1024

try:
    for epoch in range(num_epochs):
        ls_g=[]
        ls_d=[]
        for input_data,info in iterate_minibatches_with_inf(data,inform, batch_size):
            
            # Optimize D
            
            for _ in range(k_d):
                # Sample noise
                noise = Variable(torch.cat((torch.Tensor(sample_noise(len(input_data))),torch.Tensor(input_data-info)),1).cuda())
                # Do an update
                inp_data = Variable(torch.Tensor(input_data).cuda())
                data_gen = generator(noise)
                if(TASK==4):
                    #COde from here https://github.com/EmilienDupont/wgan-gp
                    alpha = torch.rand(batch_size, 1,)
                    alpha = alpha.expand_as(inp_data)
                    if use_cuda:
                        alpha = alpha.cuda()
                    interpolated = alpha * inp_data.data + (1 - alpha) * data_gen.data
                    interpolated = Variable(interpolated, requires_grad=True)
                    if use_cuda:
                        interpolated = interpolated.cuda()

                    # Calculate probability of interpolated examples
                    prob_interpolated = discriminator(interpolated)

                    # Calculate gradients of probabilities with respect to examples
                    gradients = torch.autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                                           grad_outputs=torch.ones(prob_interpolated.size()).cuda() if use_cuda else torch.ones(
                                           prob_interpolated.size()),
                                           create_graph=True, retain_graph=True)[0]

                    # Gradients have shape (batch_size, num_channels, img_width, img_height),
                    # so flatten to easily take norm per example in batch
                    gradients = gradients.view(batch_size, -1)
                   

                    # Derivatives of the gradient close to 0 can cause problems because of
                    # the square root, so manually calculate norm and add epsilon
                    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

                    # Return gradient penalty
                    penalty = 10 * ((gradients_norm - 1) ** 2).mean()
                    loss = d_loss(discriminator(data_gen), discriminator(inp_data),penalty)
                else:
                    loss = d_loss(discriminator(data_gen + Variable(torch.Tensor(info).cuda(), requires_grad=False)), discriminator(inp_data))
                ls_d.append(loss.data.cpu().numpy()[0])
                d_optimizer.zero_grad()
                loss.backward()
                d_optimizer.step()
                if TASK == 3:
                    discriminator.apply(clipper)

    
            # Optimize G
            for _ in range(k_g):
                # Sample noise
                noise = Variable(torch.cat((torch.Tensor(sample_noise(len(input_data))),torch.Tensor(input_data-info)),1).cuda())
                # Do an update
                data_gen = generator(noise)
                loss = g_loss(discriminator(data_gen + Variable(torch.Tensor(info).cuda(), requires_grad=False)))
                ls_g.append(loss.data.cpu().numpy()[0])
                g_optimizer.zero_grad()
                loss.backward()
                g_optimizer.step()
        if(epoch%10==0):
            print('generator_loss:',np.mean(ls_g),'discriminator_loss',np.mean(ls_d))
        
except KeyboardInterrupt:
    pass

In [ ]:
data_gen = generator(noise)

In [ ]:
data_gen.data.cpu().numpy()[0]

In [ ]:
pclouds1 = ae1024.decode(data_gen.data.cpu().numpy()+info) 

In [ ]:
get_plot(pclouds1,id_of_obj=4)

In [ ]:
get_plot(pclouds,id_of_obj=4)

# 3 ий уровень 

In [ ]:
generator_2048 = Generator(256, out_dim = 128).cuda()
discriminator = Discriminator(in_dim = 128).cuda()

In [ ]:
pc_ups = get_smart_upsampling(pclouds_smart1024[pclouds_smart2048.shape[0]/2:],1024,num_neighbors=4)

In [ ]:
lt = get_latent(ae=ae2048,feed_pc=pclouds_smart2048[:1000])
for i in np.arange(1000,pclouds_smart2048.shape[0],1000):
    lt =(np.concatenate((lt,get_latent(ae=ae2048,feed_pc=pclouds_smart2048[i:i+1000])),axis=0)) 
lt1024 = lt[pclouds_smart2048.shape[0]/2:]

In [ ]:
lt = get_latent(ae=ae1024,feed_pc=pc_ups[:1000])
for i in np.arange(1000,pc_ups.shape[0],1000):
    lt =(np.concatenate((lt,get_latent(ae=ae1024,feed_pc=pc_ups[i:i+1000])),axis=0)) 
lt_ups2048 = lt